In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
import random
import torch
import torch.nn as nn
from custommodels import LoadDataset, ResNet50, DenseNet121, MobileNetV2
from training import ModelTrainer

In [2]:
# to reproduce
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
db = "HAM"
root = f"../../dataset/CAI 2025/{db}/"
df_train = pd.read_csv(f"{root}/dataframe/df_distance_train.csv")
df_valid = pd.read_csv(f"{root}/dataframe/df_distance_valid.csv")
df_test = pd.read_csv(f"{root}/dataframe/df_distance_test.csv")

loader = LoadDataset(label="label", batch_size=16)
train_loader, valid_loader, test_loader = loader.create_dataloaders(df_train, df_valid, df_test)

Width: 200 Height: 150


# MobileNet

In [4]:
models = ["RES", "DENSE", "MOBILE"]

model_save_directory = f"{root}model/{models[2]}/"
if not os.path.exists(model_save_directory):
    os.makedirs(model_save_directory)
    print(f"{model_save_directory} created.")
else:
    print(f"{model_save_directory} aready exist.")
        
df_valid_filepath = f"{root}dataframe/df_valid_{models[2]}.csv"
df_test_filepath = f"{root}dataframe/df_test_{models[2]}.csv"

../../dataset/CAI 2025/HAM/model/MOBILE/ created.


In [5]:
num_class = 8
num_epochs = 30
lr = 1e-6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MobileNetV2(num_class).to(device)
trainer = ModelTrainer()
best_val_file = trainer.train(model, train_loader, valid_loader, model_save_directory, num_epochs=num_epochs, lr=lr)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/kuniko/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████████████████████████████████████████████████████████████████████████| 13.6M/13.6M [00:00<00:00, 16.1MB/s]


Outputs shape: torch.Size([16, 8])
Outputs: tensor([[-1.8585e-01,  4.2029e-02,  3.1232e-01,  3.6758e-01, -7.1135e-01,
         -6.9092e-01, -3.9796e-01,  1.0613e-02],
        [-5.5740e-01, -6.9634e-01, -3.2108e-01,  1.7646e-02,  2.6472e-01,
         -5.6385e-01, -6.0234e-01,  4.9297e-01],
        [-2.4249e-01, -1.6800e-01, -4.6518e-01,  8.2310e-02, -4.4608e-02,
         -6.0954e-01, -8.3505e-01,  3.9896e-01],
        [-7.5390e-02,  2.0289e-01,  6.3203e-02,  7.1022e-01,  1.0580e-01,
          7.3093e-04, -3.5861e-01,  3.4278e-01],
        [-3.1056e-01,  6.8551e-02,  4.7254e-02,  7.8186e-01,  2.1710e-02,
         -2.8124e-01, -1.2946e-01, -1.1571e-01],
        [-6.2787e-01, -1.4293e-01, -4.8364e-01,  2.9558e-01,  2.8897e-01,
         -8.5005e-01, -4.4024e-01,  5.8109e-01],
        [-2.8291e-01, -3.0112e-01,  9.3352e-02,  9.6116e-01, -2.5158e-01,
         -5.8081e-01, -7.4506e-02,  2.0128e-01],
        [-4.2191e-01, -1.0707e-01, -4.6003e-02,  1.0376e+00, -2.7459e-01,
         -2.3239e-01,

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [ ]:
model = ResNet50(num_class).to(device)
model.load_state_dict(torch.load(best_val_file))
test_predictions, test_outputs = trainer.evaluate(model, valid_loader)
df_valid["pred"] = test_predictions
df_valid["proba"] = test_outputs
df_valid.to_csv(df_valid_filepath, index=False)

In [ ]:
test_predictions, test_outputs = trainer.evaluate(model, test_loader)
df_test["pred"] = test_predictions
df_test["proba"] = test_outputs
df_test.to_csv(df_valid_filepath, index=False)